K-neighbors es lo recomendado para medir similaridad en datasets grandes por su eficiencia. Además, si se quiere extender la funcionalidad hacia user-item recommendation, es ideal también. En este modelo se puede definir distintas medidas de similaridad como distancia euclidiana o cosine similarity. 
Lo más importante es escoger correctamente los features a ingresar en la matriz, deben ser relevantes. -- Falta revisar si el método tfidf tiene un tope de palabras y si se puede asignar stopwords en español. 


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

# Combine relevant columns into a single 'profile'
games['profile'] = games['genre'] + " " + games['tags'] + " " + games['specs'] + " " + games['developer'] + " " + games['publisher']
games['profile'] = games['profile'].fillna('')

# Create a TF-IDF Vectorizer to transform the 'profile' into numerical data
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(games['profile'])

# Use k-NN to find similar games
knn = NearestNeighbors(n_neighbors=6, metric='cosine', algorithm='brute')  # Using 6 neighbors to include the game itself
knn.fit(tfidf_matrix)

# Create a function to get recommendations
def recommend(game_title):
    idx = games.index[games['title'] == game_title].tolist()[0]
    distances, indices = knn.kneighbors(tfidf_matrix[idx])
    
    # Exclude the first (index 0) since it'll be the game itself
    game_indices = indices[0][1:]
    return games['title'].iloc[game_indices]

# Test
print(recommend("Some Game Title"))
